In [2]:
import torch
from bert import BERTClassifier
from transformers import BertTokenizerFast

In [3]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(
    vocab_size=tokenizer.vocab_size,
    hidden_size=128,
    num_layers=4,
    num_heads=4,
    ff_dim=256,
    dropout=0.1,
).to(device)
model.load_state_dict(torch.load("output/4-0.7982-model.pth"))

<All keys matched successfully>

In [4]:
def inference(model, texts):
    model.eval()
    inputs = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=64,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()
        labels = probs.argmax(axis=1)
    return labels, probs

In [ ]:
# Run inference on example sentences
examples = [
    "This movie was fantastic!",
    "I really hated this film.",
    "The plot was engaging but the acting was bad.",
]
labels, probs = inference(model, examples)
for text, label, prob in zip(examples, labels, probs):
    print(f"Text: {text} -> Predicted Label: {label}, Probability: {prob}")

Text: This movie was fantastic! -> Predicted Label: 1, Probability: [0.4303549  0.56964505]
Text: I really hated this film. -> Predicted Label: 1, Probability: [0.3512742 0.6487258]
Text: The plot was engaging but the acting was bad. -> Predicted Label: 0, Probability: [0.988986   0.01101394]
